<a href="https://colab.research.google.com/github/pxs1990/pyspark_project/blob/main/pyspark_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e3bd3d5b724531ba0305b6b2d31a13e02adee39b0b0c2792f75888797c138207
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
sparkSession=SparkSession.builder.appName('Practise').getOrCreate()

In [51]:
df_pyspark=sparkSession.read.csv('/content/test2.csv',header=True,inferSchema=True)#inferSchema=True-->data types of each column based on csv file is inferred.
df_pyspark.show()

+-----+----+------+-----------+
| Name| Age|Salary|ExtraColumn|
+-----+----+------+-----------+
|Peter|  58| 13000|          9|
| Jack|  52| 10000|          3|
| John|  37|  7000|          5|
|  Bob|  54|  5000|          8|
| NULL|NULL|  NULL|       NULL|
|Katie|  52|  7000|          2|
|Katie|  30|  5000|          3|
| Rose|  39| 14000|          2|
| Jack|  40|  3000|          4|
| Rose|  56|  1000|          1|
| Emma|  44|  3000|          8|
| NULL|NULL|  NULL|       NULL|
|Frank|  24| 15000|          4|
|David|  27| 12000|          2|
| Noah|  53|  2000|          3|
| Rose|  22| 14000|          3|
|  Sam|  39|  1000|         10|
| Rose|  28|  3000|          8|
| Noah|  54|  6000|          5|
|Peter|  59| 15000|          5|
+-----+----+------+-----------+



In [35]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ExtraColumn: integer (nullable = true)



In [52]:
#drop the columns
df=df_pyspark.drop('Name')
df.show()

+----+------+-----------+
| Age|Salary|ExtraColumn|
+----+------+-----------+
|  58| 13000|          9|
|  52| 10000|          3|
|  37|  7000|          5|
|  54|  5000|          8|
|NULL|  NULL|       NULL|
|  52|  7000|          2|
|  30|  5000|          3|
|  39| 14000|          2|
|  40|  3000|          4|
|  56|  1000|          1|
|  44|  3000|          8|
|NULL|  NULL|       NULL|
|  24| 15000|          4|
|  27| 12000|          2|
|  53|  2000|          3|
|  22| 14000|          3|
|  39|  1000|         10|
|  28|  3000|          8|
|  54|  6000|          5|
|  59| 15000|          5|
+----+------+-----------+



In [41]:
from pyspark.sql.functions import mean

# Impute missing values in the 'Age', 'Salary', and 'ExtraColumn' columns with their respective mean values
mean_values = df.select([mean('Age').alias('mean_age'),
                                 mean('Salary').alias('mean_salary'),
                                 mean('ExtraColumn').alias('mean_extra')]).collect()

# Extract mean values
mean_age = mean_values[0]['mean_age']
mean_salary = mean_values[0]['mean_salary']
mean_extra = mean_values[0]['mean_extra']

# Fill missing values with mean values
imputed_df = df.fillna({'Age': mean_age, 'Salary': mean_salary, 'ExtraColumn': mean_extra})

# Show the DataFrame with imputed values
imputed_df.show()


+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 58| 13000|          9|
| 52| 10000|          3|
| 37|  7000|          5|
| 54|  5000|          8|
| 42|  7555|          4|
| 52|  7000|          2|
| 30|  5000|          3|
| 39| 14000|          2|
| 40|  3000|          4|
| 56|  1000|          1|
| 44|  3000|          8|
| 42|  7555|          4|
| 24| 15000|          4|
| 27| 12000|          2|
| 53|  2000|          3|
| 22| 14000|          3|
| 39|  1000|         10|
| 28|  3000|          8|
| 54|  6000|          5|
| 59| 15000|          5|
+---+------+-----------+



In [43]:
mean_values

[Row(mean_age=42.666666666666664, mean_salary=7555.555555555556, mean_extra=4.722222222222222)]

In [44]:
mean_values[0]

Row(mean_age=42.666666666666664, mean_salary=7555.555555555556, mean_extra=4.722222222222222)

In [55]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Age', 'ExtraColumn', 'Salary'],
    outputCols=["{}_imputed".format(c) for c in ['Age', 'Experience', 'Salary']]
    ).setStrategy("median")
# Add imputation cols to df
imputer.fit(df).transform(df).show()

In [56]:
# Add imputation cols to df
imputer.fit(df).transform(df).show()

+----+------+-----------+-----------+------------------+--------------+
| Age|Salary|ExtraColumn|Age_imputed|Experience_imputed|Salary_imputed|
+----+------+-----------+-----------+------------------+--------------+
|  58| 13000|          9|         58|                 9|         13000|
|  52| 10000|          3|         52|                 3|         10000|
|  37|  7000|          5|         37|                 5|          7000|
|  54|  5000|          8|         54|                 8|          5000|
|NULL|  NULL|       NULL|         40|                 4|          6000|
|  52|  7000|          2|         52|                 2|          7000|
|  30|  5000|          3|         30|                 3|          5000|
|  39| 14000|          2|         39|                 2|         14000|
|  40|  3000|          4|         40|                 4|          3000|
|  56|  1000|          1|         56|                 1|          1000|
|  44|  3000|          8|         44|                 8|        

In [57]:
#If you want to overwride cols with nan
from pyspark.ml.feature import Imputer

# Define the input and output column names for the imputer
input_cols = ['Age', 'ExtraColumn', 'Salary']
output_cols = ['Age', 'ExtraColumn', 'Salary']  # Same as input columns

# Create the Imputer
imputer = Imputer(
    inputCols=input_cols,
    outputCols=output_cols
).setStrategy("median")

# Fit the imputer and transform the DataFrame
imputed_df = imputer.fit(df).transform(df)

# Show the DataFrame with imputed values
imputed_df.show()


+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 58| 13000|          9|
| 52| 10000|          3|
| 37|  7000|          5|
| 54|  5000|          8|
| 40|  6000|          4|
| 52|  7000|          2|
| 30|  5000|          3|
| 39| 14000|          2|
| 40|  3000|          4|
| 56|  1000|          1|
| 44|  3000|          8|
| 40|  6000|          4|
| 24| 15000|          4|
| 27| 12000|          2|
| 53|  2000|          3|
| 22| 14000|          3|
| 39|  1000|         10|
| 28|  3000|          8|
| 54|  6000|          5|
| 59| 15000|          5|
+---+------+-----------+



# **Filtering:**

In [59]:
### Salary of the people less than or equal to 20000
imputed_df.filter("Salary<=10000").show()

+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 52| 10000|          3|
| 37|  7000|          5|
| 54|  5000|          8|
| 40|  6000|          4|
| 52|  7000|          2|
| 30|  5000|          3|
| 40|  3000|          4|
| 56|  1000|          1|
| 44|  3000|          8|
| 40|  6000|          4|
| 53|  2000|          3|
| 39|  1000|         10|
| 28|  3000|          8|
| 54|  6000|          5|
+---+------+-----------+



In [62]:
imputed_df.filter("Salary<=10000").select(['Salary','Age']).show()

+------+---+
|Salary|Age|
+------+---+
| 10000| 52|
|  7000| 37|
|  5000| 54|
|  6000| 40|
|  7000| 52|
|  5000| 30|
|  3000| 40|
|  1000| 56|
|  3000| 44|
|  6000| 40|
|  2000| 53|
|  1000| 39|
|  3000| 28|
|  6000| 54|
+------+---+



In [66]:

imputed_df.filter(imputed_df['Salary']<=10000).show()

+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 52| 10000|          3|
| 37|  7000|          5|
| 54|  5000|          8|
| 40|  6000|          4|
| 52|  7000|          2|
| 30|  5000|          3|
| 40|  3000|          4|
| 56|  1000|          1|
| 44|  3000|          8|
| 40|  6000|          4|
| 53|  2000|          3|
| 39|  1000|         10|
| 28|  3000|          8|
| 54|  6000|          5|
+---+------+-----------+



In [67]:
imputed_df.filter((imputed_df['Salary']<=10000) |
                  (imputed_df['Salary']>=5000)).show()

+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 58| 13000|          9|
| 52| 10000|          3|
| 37|  7000|          5|
| 54|  5000|          8|
| 40|  6000|          4|
| 52|  7000|          2|
| 30|  5000|          3|
| 39| 14000|          2|
| 40|  3000|          4|
| 56|  1000|          1|
| 44|  3000|          8|
| 40|  6000|          4|
| 24| 15000|          4|
| 27| 12000|          2|
| 53|  2000|          3|
| 22| 14000|          3|
| 39|  1000|         10|
| 28|  3000|          8|
| 54|  6000|          5|
| 59| 15000|          5|
+---+------+-----------+



In [69]:

imputed_df.filter(~(imputed_df['Salary']<=10000)).show()

+---+------+-----------+
|Age|Salary|ExtraColumn|
+---+------+-----------+
| 58| 13000|          9|
| 39| 14000|          2|
| 24| 15000|          4|
| 27| 12000|          2|
| 22| 14000|          3|
| 59| 15000|          5|
+---+------+-----------+



In [72]:
### Grouped to find the maximum salary
imputed_df.groupBy('Age').sum('Salary').show()

+---+-----------+
|Age|sum(Salary)|
+---+-----------+
| 53|       2000|
| 28|       3000|
| 27|      12000|
| 44|       3000|
| 22|      14000|
| 52|      17000|
| 40|      15000|
| 54|      11000|
| 37|       7000|
| 59|      15000|
| 39|      15000|
| 24|      15000|
| 56|       1000|
| 58|      13000|
| 30|       5000|
+---+-----------+



In [73]:

imputed_df.groupBy('Age').count().show()

+---+-----+
|Age|count|
+---+-----+
| 53|    1|
| 28|    1|
| 27|    1|
| 44|    1|
| 22|    1|
| 52|    2|
| 40|    3|
| 54|    2|
| 37|    1|
| 59|    1|
| 39|    2|
| 24|    1|
| 56|    1|
| 58|    1|
| 30|    1|
+---+-----+



In [ ]:
# Aggregate the 'Salary' column to compute the sum
sum_value_df = imputed_df.agg({'Salary': 'sum'})

# Collect the aggregated sum value
sum_value = sum_value_df.collect()[0]['sum(Salary)']

# Print the sum of the 'Salary' column
print("Sum of Salary column:", sum_value)


In [92]:
agdf=imputed_df.agg({'Salary':'sum','Age':'mean'})
agdf.show()

+-----------+--------+
|sum(Salary)|avg(Age)|
+-----------+--------+
|     148000|    42.4|
+-----------+--------+



In [93]:
agdf.collect()

[Row(sum(Salary)=148000, avg(Age)=42.4)]

In [94]:
type(agdf.collect())

list

In [96]:
agdf.collect()[0]

Row(sum(Salary)=148000, avg(Age)=42.4)

In [87]:
agdf.collect()[0]['sum(Salary)']
#OR agdf.collect()[0][0]

148000

In [98]:
agdf.collect()[0][1]

42.4